In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-09-26"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1010,2024-09-26,Austrália Nbl,06:30,South East Melbourne,Melbourne United,3.08,1.38,183.5,1.88,1.86,4.5,2.01,1.71,6FMGUiKq,0.324675,0.724638,0.531915,0.537634,0.049313,0.6,0.4,NaN,NaN,371.272,125.001338,0.336684,149.864,24.478457,0.163338,332.142,150.876,235.17,165.44,0.0,0.0,0.0,0.0,0.539050,0.007563,0.114049,0.21,0.042,49.523810,0.469560,0.4,-0.069560,-3.12,-0.624,-0.608974,0.618635,0.4,-0.218635
1011,2024-09-26,Ásia Wbl Asia Women,05:00,Fujitsu F,Cathay Life F,1.01,18.00,138.5,1.84,1.82,-24.5,1.82,1.84,M7zy7mrJ,0.990099,0.055556,0.543478,0.549451,0.045655,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,1.263939,0.007728,0.007728,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1012,2024-09-26,Ásia Wbl Asia Women,08:00,Sichuan F,Surabaya Emdee Fever F,1.83,1.83,145.5,1.83,1.83,-33.5,1.88,1.78,CznnE4kn,0.546448,0.546448,0.546448,0.546448,0.092896,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.038640,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1013,2024-09-26,Croácia Premijer Liga,13:00,Split,Kvarner,1.03,8.30,157.5,1.83,1.83,-18.5,1.98,1.70,ARg3hkhU,0.970874,0.120482,0.546448,0.546448,0.091356,0.4,0.0,NaN,NaN,273.698,285.070911,1.041553,NaN,NaN,NaN,142.254,0.000,113.85,0.00,0.0,0.0,0.0,0.0,1.101965,0.000000,0.107603,-0.49,-0.098,-0.306122,0.655793,0.8,0.144207,0.00,0.000,inf,0.000000,0.0,0.000000
1014,2024-09-26,Europa Liga Aba,14:00,Studentski Centar,Subotica,1.77,1.95,159.5,1.80,1.86,-2.5,2.04,1.69,ljxEzFUj,0.564972,0.512821,0.555556,0.537634,0.077792,0.8,0.0,NaN,NaN,152.896,22.919072,0.149900,NaN,NaN,NaN,142.876,0.000,175.20,0.00,0.0,0.0,0.0,0.0,0.068430,0.023184,0.132701,-1.52,-0.304,-2.532895,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1015,2024-09-26,Europa Liga Da Aba 2,13:00,Rabotnicki,Sutjeska,2.21,1.56,159.5,1.80,1.86,2.5,1.95,1.72,2awzIf04,0.452489,0.641026,0.555556,0.537634,0.093514,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.243830,0.023184,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1016,2024-09-26,Europa Liga Letônia,13:00,Ventspils,Rigas Zelli,2.70,1.38,164.5,1.80,1.86,4.5,2.00,1.69,2gj37XV8,0.370370,0.724638,0.555556,0.537634,0.095008,0.6,0.6,NaN,NaN,149.608,43.564069,0.291188,167.240,85.148302,0.509138,147.612,167.240,107.06,106.88,0.0,0.0,0.0,0.0,0.457540,0.023184,0.118809,0.04,0.008,212.500000,0.000000,0.0,0.000000,-2.44,-0.488,-0.778689,0.613626,0.5,-0.113626
1017,2024-09-26,Europa Liga Letônia,13:30,BC Kalev,VEF Riga,1.64,2.06,154.5,1.83,1.83,-3.5,1.95,1.72,00zcIbsG,0.609756,0.485437,0.546448,0.546448,0.095193,0.2,0.6,NaN,NaN,105.452,43.970025,0.416967,471.752,816.483393,1.730747,119.916,479.756,92.92,1932.00,0.0,1.0,0.0,0.0,0.160532,0.000000,0.088629,2.33,0.466,1.373391,0.000000,0.0,0.000000,18.18,3.636,0.291529,0.000000,0.0,0.000000
1018,2024-09-26,Europa Eurocopa Feminina,11:30,Cluj Napoca F,Iraklis F,1.21,4.06,125.5,1.85,1.81,-10.5,2.05,1.66,nR9kMpqB,0.826446,0.246305,0.540541,0.552486,0.072752,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.764802,0.015456,0.148664,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1019,2024-09-26,Filipinas Copa Dos Governadores,08:30,San Miguel Beermen,Converge FiberXers,1.33,3.21,216.5,1.88,1.86,-7.5,1.92,1.78,GrqwLA

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
